In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/통계청공모전/1. 실습용자료.txt', sep = '|', encoding = 'cp949')
test_df = pd.read_csv('/content/drive/MyDrive/통계청공모전/2. 모델개발용자료.txt', sep = '|', encoding = 'cp949')

In [3]:
'''
temp = train_df.digit_3.value_counts()[train_df.digit_3.value_counts() > 1000].index.to_list()
train_df = train_df.query('digit_3 == @temp')
'''

"\ntemp = train_df.digit_3.value_counts()[train_df.digit_3.value_counts() > 1000].index.to_list()\ntrain_df = train_df.query('digit_3 == @temp')\n"

In [4]:
train_df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [5]:
train_df= train_df.replace(np.nan, '', regex=True)
test_df= test_df.replace(np.nan, '', regex=True)

train_df['text'] = train_df['text_obj'] + " " + train_df['text_mthd'] + " " + train_df['text_deal']
test_df['text'] = test_df['text_obj'] + " " + test_df['text_mthd'] + " " + test_df['text_deal']

In [6]:
train_df['digit_3'] = train_df['digit_3'].astype(str)

In [7]:
import re 

punc = re.compile('[\!@#$%\^&\*\(\)\-\=\[\]\{\}\.,/\?~\+\'"|_:;><`┃]')

def remove_punctuation(text, lower = True):
    text = punc.sub(' ', str(text))
    text = ' '.join(text.split())
    if lower:
        text = text.lower()
    return text

train_df['text'] = train_df['text'].map(remove_punctuation)
test_df['text'] = test_df['text'].map(remove_punctuation)

In [8]:
from sklearn.preprocessing import LabelEncoder

target_list = ['digit_3']


le = LabelEncoder()
train_df['digit_3'] = le.fit_transform(train_df['digit_3'])

train_df.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0000001,S,95,221,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,120,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소 과일판매
2,id_0000003,G,46,117,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,123,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,212,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육


In [9]:
data_list = []
for q, label in zip(train_df['text'], train_df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [10]:
data_list[0]

['카센터에서 자동차부분정비 타이어오일교환', '221']

In [11]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [12]:
from torch.utils.data import Dataset, DataLoader

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
!pip install mxnet

     |████████████████████████████████| 47.3 MB 1.5 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [13]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-wam1eu0p
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-wam1eu0p
     |████████████████████████████████| 132 kB 32.5 MB/s 
     |████████████████████████████████| 344 kB 53.7 MB/s 
     |████████████████████████████████| 4.5 MB 24.6 MB/s 
     |████████████████████████████████| 1.2 MB 68.3 MB/s 
     |████████████████████████████████| 4.0 MB 71.5 MB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 77 kB 9.0 MB/s 
     |████████████████████████████████| 6.5 MB 55.0 MB/s 
     |████████████████████████████████| 79 kB 11.9 MB/s 
     |████████████████████████████████| 8.7 MB 4.5 MB/s 
     |████████████████████████████████| 127 kB 66.8 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=a96c23a6c40f

In [14]:
from kobert.pytorch_kobert import get_pytorch_kobert_model

max_len = 32
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [15]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import gluonnlp as nlp

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [16]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [17]:
device = torch.device("cuda:0")

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes= len(train_df.digit_3.value_counts()),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel


model = BERTClassifier(bertmodel,  dr_rate=0.4).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# class weights balancing
class_weights = []

for i in train_df.digit_3.unique():
    weights = (1-len(train_df[train_df.digit_3 == i])/len(train_df))
    class_weights.append(weights)

class_weights_t = torch.tensor(class_weights).cuda()

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(weight = class_weights_t) # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from tqdm import tqdm, tqdm_notebook

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} texst acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))
    try:
        torch.save(model.state_dict(), f'/content/drive/MyDrive/통계청공모전/kobert_digit3_e{e + 1}.pt')
        print(f'model save epoch{e + 1}')
    except:
        print('model save err')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/25000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 5.493335247039795 train acc 0.0
epoch 1 batch id 201 loss 5.472526550292969 train acc 0.005286069651741294
epoch 1 batch id 401 loss 5.322941303253174 train acc 0.008494389027431421
epoch 1 batch id 601 loss 5.176341533660889 train acc 0.02839018302828619
epoch 1 batch id 801 loss 5.0244245529174805 train acc 0.05469725343320849
epoch 1 batch id 1001 loss 4.730397701263428 train acc 0.07573676323676323
epoch 1 batch id 1201 loss 4.550289630889893 train acc 0.09601373855120733
epoch 1 batch id 1401 loss 4.083897113800049 train acc 0.1183083511777302
epoch 1 batch id 1601 loss 3.7191507816314697 train acc 0.1398930356027483
epoch 1 batch id 1801 loss 4.228090763092041 train acc 0.16272209883398112
epoch 1 batch id 2001 loss 3.4924118518829346 train acc 0.18443903048475763
epoch 1 batch id 2201 loss 2.635349988937378 train acc 0.20737732848705134
epoch 1 batch id 2401 loss 2.9527788162231445 train acc 0.23147907122032488
epoch 1 batch id 2601 loss 2.618334770202636

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 batch id 1 loss 0.27688753604888916 train acc 0.90625
epoch 2 batch id 201 loss 0.31752729415893555 train acc 0.896610696517413
epoch 2 batch id 401 loss 0.4125591516494751 train acc 0.8975218204488778
epoch 2 batch id 601 loss 0.34009411931037903 train acc 0.8978265391014975
epoch 2 batch id 801 loss 0.5553493499755859 train acc 0.8993055555555556
epoch 2 batch id 1001 loss 0.1336354911327362 train acc 0.8997564935064936
epoch 2 batch id 1201 loss 0.2708672881126404 train acc 0.9003434637801832
epoch 2 batch id 1401 loss 0.21597544848918915 train acc 0.8993799072091363
epoch 2 batch id 1601 loss 0.2536706328392029 train acc 0.8989108369768894
epoch 2 batch id 1801 loss 0.714570939540863 train acc 0.8996217379233759
epoch 2 batch id 2001 loss 0.34054747223854065 train acc 0.899753248375812
epoch 2 batch id 2201 loss 0.37792375683784485 train acc 0.8995485006815084
epoch 2 batch id 2401 loss 0.19380898773670197 train acc 0.8998203873386089
epoch 2 batch id 2601 loss 0.4810886979

  0%|          | 0/6250 [00:00<?, ?it/s]

epoch 2 texst acc 0.91231
model save epoch2


  0%|          | 0/25000 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.22676384449005127 train acc 0.9375
epoch 3 batch id 201 loss 0.2728342115879059 train acc 0.9124689054726368
epoch 3 batch id 401 loss 0.3013577163219452 train acc 0.9161471321695761
epoch 3 batch id 601 loss 0.10434923321008682 train acc 0.9169093178036606
epoch 3 batch id 801 loss 0.14235332608222961 train acc 0.9176029962546817
epoch 3 batch id 1001 loss 0.06519326567649841 train acc 0.9179882617382618
epoch 3 batch id 1201 loss 0.2309321165084839 train acc 0.9179589925062448
epoch 3 batch id 1401 loss 0.1319805085659027 train acc 0.9173358315488936
epoch 3 batch id 1601 loss 0.3488009572029114 train acc 0.9168293254216114
epoch 3 batch id 1801 loss 0.40805327892303467 train acc 0.9172508328706275
epoch 3 batch id 2001 loss 0.4454127252101898 train acc 0.9169165417291354
epoch 3 batch id 2201 loss 0.22650930285453796 train acc 0.9168417764652431
epoch 3 batch id 2401 loss 0.22244210541248322 train acc 0.9167013744273219
epoch 3 batch id 2601 loss 0.40208342

In [ ]:
test_data = test_df[['text']]
test_label = [0 for i in range(len(test_df))]
test_data['label'] = test_label

test_data = [list(test_data.iloc[i]) for i in range(len(test_data))]
print(test_data[:10])
test_data

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/통계청공모전/kobert_digit3_e10.pt')

In [ ]:
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
predictions =[]
logits = []
with torch.no_grad():
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        output = model(token_ids, valid_length, segment_ids)
        logits.extend(output.detach().cpu().numpy())
        _, max_indices = torch.max(output, 1)
        predictions.extend(max_indices.squeeze(0).detach().cpu().numpy())

In [ ]:
final = le.inverse_transform(predictions)

In [ ]:
test_df['digit_3'] = final
test_df

In [ ]:
test_df['digit_3'] = test_df['digit_3'].astype(int)

In [ ]:
test_df.info()

In [ ]:
test_df.to_csv('/content/drive/MyDrive/kobert_digit3_e10.csv')

In [ ]:
np.save('/content/drive/MyDrive/통계청공모전/kobert_logits_e10', logits)

In [ ]:
l = np.load('/content/drive/MyDrive/통계청공모전/kobert_logits_e10.npy')
np.array_equal(logits, l)